In [1]:
from metaflow import Step, Flow, Run

import utilities

In [2]:
run_id = '1620937510032366'

In [3]:
runs = Flow('Train/start')


In [4]:
steps = Step(f'Train/{run_id}/split_folds')


In [5]:
data = list(steps)[0].data
data

<MetaflowData: fold, train_validation_index, categorical_features, folds, name, tuning_grid, df, k_fold, numeric_features, test_index, hyperparameters>

In [6]:
data.df

,target,num_0,num_1,num_2,num_3,num_4,num_5,num_6,num_7,num_8,num_9,cat
0,-226.465449,-1.039309,-0.533254,0.006352,-0.130216,-0.672371,-1.227693,-1.605115,0.313087,1.709311,1.486217,type_0
1,53.328963,0.906268,1.112101,-0.816500,0.461619,0.883569,1.125719,-0.993897,0.999854,-1.919401,-1.137031,type_0
2,152.631640,0.334137,0.320004,-0.248267,-0.317444,0.834343,1.381073,0.901058,-0.655725,0.340868,-1.481551,type_0
3,-418.551943,0.250441,-1.215110,-1.562450,0.162566,-1.630155,-0.449801,-1.033361,-0.671750,-1.331549,-0.979638,type_0
4,7.570045,-1.440993,-0.388298,-0.431737,0.518420,-0.405904,-0.785488,1.008090,-0.695019,1.885108,-0.913755,type_0
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-203.630169,-0.006985,-0.343568,-1.321507,0.911448,-0.397213,-0.928320,0.538747,-0.310263,-0.166743,-1.960105,type_4
49996,-83.068843,0.190101,0.078127,-0.023049,-0.773157,0.170425,-0.746559,-1.705857,-0.488374,-0.922338,-0.310841,type_4
49997,69.608942,-0.333286,0.906969,0.031803,0.147185,-0.310786,-0.359339,-2.461471,0.010472,0.366817,-0.055318,type_4
49998,-133.474475,0.822524,0.290625,-0.628422,0.444869,-1.667508,-1.604436,0.001591,-0.784573,-1.157536,-1.121551,type_4


In [7]:
data.hyperparameters

{'max_depth': 1,
 'learning_rate': 0.1,
 '__model': 'utilities.LightGBMHandler',
 '__fit_kwargs': {'model__eval_metric': 'mse',
  'model__early_stopping_rounds': 10,
  'model__verbose': 1}}

In [8]:
data.fold

0

In [10]:
import pickle
from scipy import stats
import numpy as np
from metaflow import FlowSpec, step
from sklearn.model_selection import train_test_split, ParameterGrid, KFold
from sklearn.metrics import r2_score
import config
import utilities


hyperparameters = data.hyperparameters
fit_kwargs = hyperparameters['__fit_kwargs']

X = data.df.loc[data.train_validation_index, :]
y = data.df.loc[data.train_validation_index, 'target']
if data.k_fold is not None:
    train, test = list(data.k_fold.split(X))[data.fold]
else:
    train, test = train_test_split(list(range(X.shape[0])), test_size=0.2)
X_train = X.iloc[train, :]
y_train = y.iloc[train]
X_test = X.iloc[test, :]
y_test = y.iloc[test]

if hyperparameters['__model'] == 'utilities.KerasHandler':
    pipeline = utilities.build_pipeline(
        numeric_features=data.numeric_features,
        categorical_features=data.categorical_features,
    )
    X_train_transformed = pipeline.fit_transform(X_train)
    hyperparameters.update({
        'input_dim': X_train_transformed.shape[1],
    })

model_handler_class = utilities.import_object_from_string(hyperparameters['__model'])

pipeline = utilities.build_pipeline(
    numeric_features=data.numeric_features,
    categorical_features=data.categorical_features,
    model=model_handler_class(**{k: v for k, v in hyperparameters.items() if not k.startswith('__')}),
)
pipeline.fit(X_train, y_train, model__validation_data=(X_test, y_test), **fit_kwargs)


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: cat

In [11]:
X_train

,target,num_0,num_1,num_2,num_3,num_4,num_5,num_6,num_7,num_8,num_9,cat
19475,-12.447417,-1.595766,-0.817986,0.306061,-0.080776,-1.130654,-0.110107,-0.044770,1.228896,-1.152385,-1.156432,type_1
15894,-84.681328,0.178626,1.515139,0.137979,1.666983,1.993133,-0.024486,-1.916808,-0.700803,-0.985502,-0.289782,type_1
40857,-129.842598,0.150418,-1.889437,-0.877766,-1.179731,1.342296,0.398567,-0.062284,0.902770,-0.113951,0.637760,type_4
43786,-34.449808,-0.836572,0.071820,-0.373441,1.582668,-0.333944,-1.037247,0.291401,0.321876,0.086945,-0.687993,type_4
2445,-255.087810,-0.239328,-1.139918,-1.092081,-0.776154,-0.666790,1.353597,-0.854890,0.703368,-0.950649,2.134648,type_0
...,...,...,...,...,...,...,...,...,...,...,...,...
29257,-53.320370,0.692571,0.501303,-0.125529,0.271168,-0.548505,0.317498,0.266075,0.194255,-0.704436,-1.058060,type_2
8274,-29.106288,0.100548,0.477510,-0.464388,0.448110,0.611569,-1.112819,-1.251741,0.836626,-0.026251,-1.313781,type_0
11664,17.624257,-0.883659,1.345242,-1.043574,0.963145,0.109741,2.038063,1.976235,0.544350,0.550535,0.062360,type_1
34791,-103.037162,0.053052,2.062208,-1.739565,1.268021,0.247431,1.142334,-0.037998,0.762926,-0.773303,0.629002,type_3
